In [23]:

import numpy as np
import pandas as pd
from scipy.stats.mstats import winsorize

import warnings
warnings.filterwarnings('ignore')

# Washington State HDMA 2016

In [24]:
file = "raw_data/Washington_State_HDMA-2016.csv"
data_original = pd.read_csv(file, decimal=',')

In [25]:
data = data_original.copy()
data.shape

(466566, 47)

In [26]:
# --- Data manipulation ---


def clean_data(data: pd.DataFrame):
    # Define list of columns to filter for "Information not provided" values
    filter_cols = ['applicant_ethnicity_name', 'applicant_race_name_1', 'applicant_sex_name',
                   'co_applicant_ethnicity_name', 'co_applicant_race_name_1', 'co_applicant_sex_name']

    # Filter for rows where "action_taken_name" is not equal to certain values
    data = data.loc[~data['action_taken_name'].isin(['Application withdrawn by applicant', 'Loan purchased by the institution', 'File closed for incompleteness'])]

    # Filter for rows where values in specified columns are not equal to "Information not provided"
    for col in filter_cols:
        data = data.loc[~data[col].isin(['Information not provided by applicant in mail, Internet, or telephone application'])]

        # Create a dictionary of county to region mappings
    county_to_region = {
        'Whatcom County': 'Northern Cascades','Skagit County': 'Northern Cascades','Snohomish County': 'Northern Cascades',
        'King County': 'Western Region','Pierce County': 'Western Region','Kitsap County': 'Western Region',
        'Island County': 'Western Region','San Juan County': 'Western Region','Jefferson County': 'Western Region',
        'Clallam County': 'Western Region','Mason County': 'Olympic Peninsula','Clark County': 'Southwest Washington',
        'Cowlitz County': 'Southwest Washington','Wahkiakum County': 'Southwest Washington','Skamania County': 'Southwest Washington',
        'Adams County': 'Eastern Washington','Asotin County': 'Eastern Washington','Benton County': 'Eastern Washington',
        'Chelan County': 'Eastern Washington','Columbia County': 'Eastern Washington','Douglas County': 'Eastern Washington',
        'Ferry County': 'Eastern Washington','Franklin County': 'Eastern Washington','Garfield County': 'Eastern Washington',
        'Grant County': 'Eastern Washington','Kittitas County': 'Eastern Washington','Klickitat County': 'Eastern Washington',
        'Lincoln County': 'Eastern Washington','Okanogan County': 'Eastern Washington','Pend Oreille County': 'Eastern Washington',
        'Spokane County': 'Eastern Washington','Stevens County': 'Eastern Washington','Walla Walla County': 'Eastern Washington',
        'Whitman County': 'Eastern Washington','Yakima County': 'Eastern Washington','Thurston County':'Western Region',
        'Lewis County': 'Western Region','Grays Harbor County': 'Western Region','Pacific County': 'Southwest Washington',
        'Seattle, Bellevue, Everett': 'Western Region','Tacoma, Lakewood':'Northern Cascades','Portland, Vancouver, Hillsboro':'Northern Cascades',
        'Spokane, Spokane Valley':'Eastern Washington'
    }

    # Add a new column to your dataframe containing the region for each county
    data['region'] = data['county_name'].map(county_to_region)

    # Drop rows with missing values in "county_name" column
    data = data.dropna(subset=['county_name'])

    # Create new column "loan_status" based on "action_taken_name"
    data['loan_status'] = np.where(data['action_taken_name'] == 'Loan originated', 'approved', 'not approved')

    # Drop irrelevant columns
    drop_cols = ['applicant_race_name_5', 'applicant_race_name_4', 'applicant_race_name_3',
                 'applicant_race_name_2', 'co_applicant_race_name_5', 'co_applicant_race_name_4',
                 'co_applicant_race_name_3', 'co_applicant_race_name_2', 'denial_reason_name_3',
                 'denial_reason_name_2', 'denial_reason_name_1', 'rate_spread', 'edit_status_name',
                 'state_abbr', 'respondent_id', 'agency_abbr', 'as_of_year', 'application_date_indicator',
                 'state_name', 'sequence_number', 'census_tract_number', 'action_taken_name', 'purchaser_type_name',
                 'county_name','msamd_name']

    data = data.drop(columns=drop_cols)

    # Winsorize numeric columns
    data.select_dtypes(exclude=['object']).apply(lambda x: winsorize(x, limits=[0.05, 0.05]), axis=0, raw=True)

    # create a list of columns to be converted
    cols_to_convert = [
        'tract_to_msamd_income','population','minority_population','number_of_owner_occupied_units',
        'number_of_1_to_4_family_units','loan_amount_000s','hud_median_family_income','applicant_income_000s'
    ]

    # use the astype() method to convert the dtype of columns
    data[cols_to_convert] = data[cols_to_convert].astype('float')

    print("\n✅ data cleaned")

    return data

In [27]:
data = clean_data(data)


✅ data cleaned


In [28]:
data

,tract_to_msamd_income,population,minority_population,number_of_owner_occupied_units,number_of_1_to_4_family_units,loan_amount_000s,hud_median_family_income,applicant_income_000s,property_type_name,preapproval_name,...,hoepa_status_name,co_applicant_sex_name,co_applicant_race_name_1,co_applicant_ethnicity_name,applicant_sex_name,applicant_race_name_1,applicant_ethnicity_name,agency_name,region,loan_status
1,83.370003,4915.0,23.990000,1268.0,1777.0,240.0,57900.0,42.0,One-to-four family dwelling (other than manufa...,Not applicable,...,Not a HOEPA loan,No co-applicant,No co-applicant,No co-applicant,Male,White,Hispanic or Latino,Department of Housing and Urban Development,Eastern Washington,approved
2,91.129997,5075.0,11.820000,1136.0,1838.0,241.0,73300.0,117.0,One-to-four family dwelling (other than manufa...,Not applicable,...,Not a HOEPA loan,Female,White,Not Hispanic or Latino,Male,White,Not Hispanic or Latino,Department of Housing and Urban Development,Southwest Washington,approved
4,162.470001,5183.0,10.500000,1705.0,2104.0,417.0,78100.0,114.0,One-to-four family dwelling (other than manufa...,Not applicable,...,Not a HOEPA loan,Male,White,Not Hispanic or Latino,Female,White,Not Hispanic or Latino,Federal Deposit Insurance Corporation,Western Region,approved
5,119.629997,4775.0,18.889999,954.0,1332.0,206.0,72300.0,NaN,One-to-four family dwelling (other than manufa...,Not applicable,...,Not a HOEPA loan,No co-applicant,No co-applicant,No co-applicant,Male,White,Not Hispanic or Latino,Office of the Comptroller of the Currency,Western Region,approved
6,103.209999,4745.0,33.070000,1337.0,1442.0,356.0,90300.0,NaN,One-to-four family dwelling (other than manufa...,Not applicable,...,Not a HOEPA loan,No co-applicant,No co-applicant,No co-applicant,Male,White,Not Hispanic or Latino,Department of Housing and Urban Development,Northern Cascades,approved
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
452825,109.730003,5035.0,18.830000,1249.0,1567.0,280.0,73300.0,115.0,One-to-four family dwelling (other than manufa...,Preapproval was requested,...,Not a HOEPA loan,Female,White,Not Hispanic or Latino,Male,White,Not Hispanic or Latino,Consumer Financial Protection Bureau,Southwest Washington,not approved
452834,59.049999,3608.0,17.959999,579.0,1150.0,300.0,72300.0,85.0,One-to-four family dwelling (other than manufa...,Preapproval was requested,...,Not a HOEPA loan,No co-applicant,No co-applicant,No co-applicant,Male,White,Not Hispanic or Latino,Department of Housing and Urban Development,Western Region,not approved
464253,101.290001,8722.0,39.070000,2012.0,2432.0,319.0,61400.0,74.0,One-to-four family dwelling (other than manufa...,Preapproval was requested,...,Not a HOEPA loan,Female,White,Not Hispanic or Latino,Male,White,Not Hispanic or Latino,Department of Housing and Urban Development,Northern Cascades,not approved
466564,88.690002,4725.0,45.880001,1188.0,1686.0,255.0,72300.0,77.0,One-to-four family dwelling (other than manufa...,Preapproval was requested,...,Not a HOEPA loan,Female,White,Not Hispanic or Latino,Male,White,Not Hispanic or Latino,Department of Housing and Urban Development,Western Region,not approved


In [29]:
data.shape

(283225, 24)

In [5]:
data.columns.to_list()

['tract_to_msamd_income',
 'rate_spread',
 'population',
 'minority_population',
 'number_of_owner_occupied_units',
 'number_of_1_to_4_family_units',
 'loan_amount_000s',
 'hud_median_family_income',
 'applicant_income_000s',
 'state_name',
 'state_abbr',
 'sequence_number',
 'respondent_id',
 'purchaser_type_name',
 'property_type_name',
 'preapproval_name',
 'owner_occupancy_name',
 'msamd_name',
 'loan_type_name',
 'loan_purpose_name',
 'lien_status_name',
 'hoepa_status_name',
 'edit_status_name',
 'denial_reason_name_3',
 'denial_reason_name_2',
 'denial_reason_name_1',
 'county_name',
 'co_applicant_sex_name',
 'co_applicant_race_name_5',
 'co_applicant_race_name_4',
 'co_applicant_race_name_3',
 'co_applicant_race_name_2',
 'co_applicant_race_name_1',
 'co_applicant_ethnicity_name',
 'census_tract_number',
 'as_of_year',
 'application_date_indicator',
 'applicant_sex_name',
 'applicant_race_name_5',
 'applicant_race_name_4',
 'applicant_race_name_3',
 'applicant_race_name_2',
 '

## Cleaning Data

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 466566 entries, 0 to 466565
Data columns (total 47 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   tract_to_msamd_income           465951 non-null  object 
 1   rate_spread                     8638 non-null    object 
 2   population                      465956 non-null  float64
 3   minority_population             465956 non-null  object 
 4   number_of_owner_occupied_units  465944 non-null  float64
 5   number_of_1_to_4_family_units   465955 non-null  float64
 6   loan_amount_000s                466566 non-null  int64  
 7   hud_median_family_income        465960 non-null  float64
 8   applicant_income_000s           404533 non-null  float64
 9   state_name                      466566 non-null  object 
 10  state_abbr                      466566 non-null  object 
 11  sequence_number                 466566 non-null  int64  
 12  respondent_id   

### Check for missing values: Use functions such as isna(), isnull(), or notnull() to detect missing values in your dataset.

In [7]:
null_df = pd.concat([data.isnull().sum().sort_values(ascending=False),
                     data.notnull().sum().sort_values(ascending=True)],
                    axis=1, keys=['is_null', 'not_null'])

null_df['percent_null'] = round((null_df['is_null'] / len(data)) * 100, 2)

null_df

,is_null,not_null,percent_null
co_applicant_race_name_5,466552,14,100.00
co_applicant_race_name_4,466545,21,100.00
applicant_race_name_5,466520,46,99.99
applicant_race_name_4,466498,68,99.99
co_applicant_race_name_3,466461,105,99.98
applicant_race_name_3,466269,297,99.94
denial_reason_name_3,465320,1246,99.73
co_applicant_race_name_2,464704,1862,99.60
applicant_race_name_2,462088,4478,99.04
denial_reason_name_2,459820,6746,98.55


In [8]:
cols_drop = ['applicant_race_name_5', 'applicant_race_name_4','applicant_race_name_3','applicant_race_name_2',
            'co_applicant_race_name_5','co_applicant_race_name_4','co_applicant_race_name_3','co_applicant_race_name_2',
            'denial_reason_name_3','denial_reason_name_2','denial_reason_name_1','rate_spread','edit_status_name',
            'respondent_id', 'state_abbr', 'agency_abbr','as_of_year', 'state_name', 'sequence_number'
           ]

In [9]:
data = data[data.columns.difference(cols_drop)]

In [10]:
print(f'data shape: {data.shape}')

data.isnull().sum().sort_values(ascending=False)

data shape: (466566, 28)


applicant_income_000s             62033
msamd_name                        38274
number_of_owner_occupied_units      622
tract_to_msamd_income               615
number_of_1_to_4_family_units       611
population                          610
minority_population                 610
census_tract_number                 606
hud_median_family_income            606
county_name                         367
loan_purpose_name                     0
purchaser_type_name                   0
property_type_name                    0
preapproval_name                      0
owner_occupancy_name                  0
loan_type_name                        0
action_taken_name                     0
loan_amount_000s                      0
agency_name                           0
hoepa_status_name                     0
co_applicant_sex_name                 0
co_applicant_race_name_1              0
co_applicant_ethnicity_name           0
application_date_indicator            0
applicant_sex_name                    0


### Check for duplicates: Use the duplicated() function to detect duplicate rows in your dataset.

In [11]:
data[data.duplicated()]

,action_taken_name,agency_name,applicant_ethnicity_name,applicant_income_000s,applicant_race_name_1,applicant_sex_name,application_date_indicator,census_tract_number,co_applicant_ethnicity_name,co_applicant_race_name_1,...,minority_population,msamd_name,number_of_1_to_4_family_units,number_of_owner_occupied_units,owner_occupancy_name,population,preapproval_name,property_type_name,purchaser_type_name,tract_to_msamd_income
57,Loan originated,Consumer Financial Protection Bureau,Not applicable,NaN,Not applicable,Not applicable,0,404.12,No co-applicant,No co-applicant,...,16.56999969,"Portland, Vancouver, Hillsboro - OR, WA",1506.0,1207.0,Not owner-occupied as a principal dwelling,4333.0,Not applicable,One-to-four family dwelling (other than manufa...,Loan was not originated or was not sold in cal...,112.5599976
147,Loan originated,Consumer Financial Protection Bureau,Not applicable,NaN,Not applicable,Not applicable,0,404.12,No co-applicant,No co-applicant,...,16.56999969,"Portland, Vancouver, Hillsboro - OR, WA",1506.0,1207.0,Not owner-occupied as a principal dwelling,4333.0,Not applicable,One-to-four family dwelling (other than manufa...,Loan was not originated or was not sold in cal...,112.5599976
694,Loan originated,Consumer Financial Protection Bureau,Not applicable,NaN,Not applicable,Not applicable,0,108.03,No co-applicant,No co-applicant,...,14.73999977,"Kennewick, Richland - WA",2137.0,2037.0,Not owner-occupied as a principal dwelling,6690.0,Not applicable,One-to-four family dwelling (other than manufa...,Loan was not originated or was not sold in cal...,171.8899994
1762,Loan originated,Consumer Financial Protection Bureau,Not applicable,NaN,Not applicable,Not applicable,0,411.1,No co-applicant,No co-applicant,...,20.22999954,"Portland, Vancouver, Hillsboro - OR, WA",1816.0,1428.0,Not owner-occupied as a principal dwelling,5849.0,Not applicable,One-to-four family dwelling (other than manufa...,Loan was not originated or was not sold in cal...,87.29000092
1817,Loan originated,Consumer Financial Protection Bureau,Not applicable,NaN,Not applicable,Not applicable,0,411.1,No co-applicant,No co-applicant,...,20.22999954,"Portland, Vancouver, Hillsboro - OR, WA",1816.0,1428.0,Not owner-occupied as a principal dwelling,5849.0,Not applicable,One-to-four family dwelling (other than manufa...,Loan was not originated or was not sold in cal...,87.29000092
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
466338,Loan purchased by the institution,Consumer Financial Protection Bureau,Not applicable,244.0,Not applicable,Not applicable,2,8.03,Not applicable,Not applicable,...,17.67000008,Bellingham - WA,2315.0,1709.0,Not owner-occupied as a principal dwelling,6275.0,Not applicable,One-to-four family dwelling (other than manufa...,Freddie Mac (FHLMC),122.0999985
466463,Loan purchased by the institution,Consumer Financial Protection Bureau,Not applicable,NaN,Not applicable,Not applicable,2,9.02,Not applicable,Not applicable,...,13.18000031,Bellingham - WA,2216.0,1752.0,Owner-occupied as a principal dwelling,5833.0,Not applicable,One-to-four family dwelling (other than manufa...,Freddie Mac (FHLMC),129.25
466492,Loan purchased by the institution,Consumer Financial Protection Bureau,Not applicable,NaN,Not applicable,Not applicable,2,1,Not applicable,Not applicable,...,20.92000008,Bellingham - WA,2537.0,2033.0,Owner-occupied as a principal dwelling,8998.0,Not applicable,One-to-four family dwelling (other than manufa...,Freddie Mac (FHLMC),100.2900009
466493,Loan purchased by the institution,Consumer Financial Protection Bureau,Not applicable,NaN,Not applicable,Not applicable,2,103.03,Not applicable,Not applicable,...,16.19000053,Bellingham - WA,1440.0,1167.0,Owner-occupied as a principal dwelling,5103.0,Not applicable,One-to-four family dwelling (other than manufa...,Freddie Mac (FHLMC),97.12000275


In [12]:
print(f'Duplicates with respondant ID: {data_original.duplicated().sum()}')
print(f'Duplicates without respondant ID: {data.duplicated().sum()}')

Duplicates with respondant ID: 0
Duplicates without respondant ID: 1652


- Same applicant applied twice as they had a different respondent ID

### Check for inconsistent values: Look for inconsistent values across different columns in your dataset.

In [13]:
# for i in data:
#     print("\n",i,":\n",data[i].unique())

### Handle inconsistent values: Deleting the rows with inconsistent values.

In [14]:
data = data[~data['action_taken_name'].isin(['Application withdrawn by applicant', 'Loan purchased by the institution', 'File closed for incompleteness'])]
data = data[~data['applicant_ethnicity_name'].isin(['Information not provided by applicant in mail, Internet, or telephone application'])]
data = data[~data['applicant_race_name_1'].isin(['Information not provided by applicant in mail, Internet, or telephone application'])]
data = data[~data['applicant_sex_name'].isin(['Information not provided by applicant in mail, Internet, or telephone application'])]
data = data[~data['co_applicant_ethnicity_name'].isin(['Information not provided by applicant in mail, Internet, or telephone application'])]
data = data[~data['co_applicant_race_name_1'].isin(['Information not provided by applicant in mail, Internet, or telephone application'])]
data = data[~data['co_applicant_sex_name'].isin(['Information not provided by applicant in mail, Internet, or telephone application'])]
data =  data.dropna(subset=['county_name'])

In [15]:
print(f'Data Shape: {data.shape}')

Data Shape: (283225, 28)


### Check for outliers: Look for data points that are significantly different from the other data points in your dataset.

In [16]:
data_original.describe()

,population,number_of_owner_occupied_units,number_of_1_to_4_family_units,loan_amount_000s,hud_median_family_income,applicant_income_000s,sequence_number,as_of_year,application_date_indicator
count,465956.000000,465944.00000,465955.000000,466566.000000,465960.000000,404533.000000,4.665660e+05,466566.0,466566.000000
mean,5288.219117,1389.05978,1822.137288,298.260936,76797.148468,112.984713,1.126340e+05,2016.0,0.207285
std,1665.350069,506.75606,682.031693,760.741532,12959.051933,125.115972,2.020830e+05,0.0,0.609593
min,5.000000,10.00000,10.000000,1.000000,48700.000000,1.000000,1.000000e+00,2016.0,0.000000
25%,4124.000000,1032.00000,1395.000000,172.000000,65800.000000,60.000000,5.121000e+03,2016.0,0.000000
50%,5173.000000,1359.00000,1734.000000,248.000000,73300.000000,89.000000,2.647200e+04,2016.0,0.000000
75%,6342.000000,1705.00000,2171.000000,350.000000,90300.000000,133.000000,1.095258e+05,2016.0,0.000000
max,13025.000000,2997.00000,5893.000000,99999.000000,90300.000000,9999.000000,1.241780e+06,2016.0,2.000000


In [17]:
df_selected = data.drop(data.select_dtypes('object').columns.tolist(),axis=1)

for i in df_selected:
    data[i] = winsorize(data[i], limits=[0.05, 0.05])

data.describe()

,applicant_income_000s,application_date_indicator,hud_median_family_income,loan_amount_000s,number_of_1_to_4_family_units,number_of_owner_occupied_units,population
count,260191.000000,283225.0,283225.000000,283225.000000,283225.000000,283225.000000,283225.000000
mean,111.604275,0.0,76810.852502,269.338254,1806.623713,1387.604226,5257.295631
std,117.842795,0.0,12719.400941,134.807431,564.047536,468.496248,1474.125310
min,34.000000,0.0,55600.000000,68.000000,910.000000,597.000000,2788.000000
25%,59.000000,0.0,65800.000000,171.000000,1397.000000,1032.000000,4125.000000
50%,88.000000,0.0,73300.000000,246.000000,1735.000000,1361.000000,5175.000000
75%,131.000000,0.0,90300.000000,348.000000,2175.000000,1709.000000,6342.000000
max,6977.000000,0.0,90300.000000,581.000000,2976.000000,2307.000000,7983.000000


- we're replacing the lowest 5% of values with the value at the 5th percentile, and the highest 5% of values with the value at the 95th percentile, using the 'winsorize' function from 'scipy.stats.mstats'.

In [18]:
data.isnull().sum()

action_taken_name                     0
agency_name                           0
applicant_ethnicity_name              0
applicant_income_000s             23034
applicant_race_name_1                 0
applicant_sex_name                    0
application_date_indicator            0
census_tract_number                  68
co_applicant_ethnicity_name           0
co_applicant_race_name_1              0
co_applicant_sex_name                 0
county_name                           0
hoepa_status_name                     0
hud_median_family_income              0
lien_status_name                      0
loan_amount_000s                      0
loan_purpose_name                     0
loan_type_name                        0
minority_population                  70
msamd_name                        23339
number_of_1_to_4_family_units         0
number_of_owner_occupied_units        0
owner_occupancy_name                  0
population                            0
preapproval_name                      0


KeyError: "['applicant_race_name_5', 'applicant_race_name_4', 'applicant_race_name_3', 'applicant_race_name_2', 'co_applicant_race_name_5', 'co_applicant_race_name_4', 'co_applicant_race_name_3', 'co_applicant_race_name_2', 'denial_reason_name_3', 'denial_reason_name_2', 'denial_reason_name_1', 'rate_spread', 'edit_status_name', 'state_abbr', 'respondent_id', 'agency_abbr', 'as_of_year', 'state_name', 'sequence_number'] not found in axis"